In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf

In [2]:
tf.__version__

'2.16.1'

In [18]:
batch_size = 128
img_height = 48
img_width = 48
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
import pathlib
data_dir_train = pathlib.Path('./images/images/train')
data_dir_val = pathlib.Path('./images/images/validation')

In [19]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale'
  )

Found 28821 files belonging to 7 classes.


In [20]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_val,
  image_size=(img_height, img_width),
  label_mode='categorical',
  batch_size=batch_size,
  color_mode='grayscale'
  )

Found 7066 files belonging to 7 classes.


In [21]:
train_ds.class_names

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, Rescaling, RandomZoom, RandomContrast

rescaling = Sequential([
    Rescaling(1./255),
])

preprocessing_seq = Sequential([
    rescaling,
    RandomFlip(),
    RandomRotation(0.2),
])

In [23]:
new_train_ds = train_ds.map(
    lambda x, y: (preprocessing_seq(x, training=True), y)
)

In [24]:
new_val_ds = val_ds.map(
    lambda x, y: (rescaling(x), y)
)

In [25]:
new_train_ds.prefetch(buffer_size=AUTOTUNE)
new_val_ds.prefetch(buffer_size=AUTOTUNE)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>

In [11]:
num_classes = 7

In [26]:
from tensorflow.keras.layers import Conv2D, Input, Dropout, Dense, Lambda, MaxPool2D, Flatten, BatchNormalization
from tensorflow.nn import local_response_normalization

alexnet = Sequential([
    Input(shape=(48, 48, 1)),
    Conv2D(filters=96, kernel_size=11, strides=4, padding='same', activation='relu'),
    Lambda(local_response_normalization),
    MaxPool2D(pool_size=3, strides=2, padding='same'),
    BatchNormalization(),

    Conv2D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu'),
    Lambda(local_response_normalization),
    MaxPool2D(pool_size=3, strides=2,  padding='same'),
    BatchNormalization(),

    Conv2D(filters=384, kernel_size=3, strides=1, padding='same', activation='relu'),
    Conv2D(filters=384, kernel_size=3, strides=1, padding='same', activation='relu'),
    Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Flatten(),
    Dense(units=4096, activation='relu'),
    Dense(units=4096, activation='relu'),
    Dropout(0.2),
    Dense(units=num_classes, activation='softmax')
])

In [27]:
alexnet.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 96)     │        11,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_2 (Lambda)               │ (None, 12, 12, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 6, 6, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 6, 6, 256)      │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_3 (Lambda)               │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 3, 3, 384)      │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 3, 3, 384)      │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 3, 3, 256)      │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │     9,441,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │        28,679 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,977,671 (114.36 MB)

 Trainable params: 29,976,455 (114.35 MB)

 Non-trainable params: 1,216 (4.75 KB)

In [34]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import AUC, CategoricalAccuracy

alexnet.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', AUC(curve='ROC')])

In [35]:
from tensorflow.keras.callbacks import ModelCheckpoint

epochs = 100

alexnet.fit(
    new_train_ds, 
    validation_data=new_val_ds,
    epochs=epochs,
    callbacks=[ModelCheckpoint('alexnet.keras', monitor='val_accuracy', save_best_only=True, mode='max')]
)

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 85s 360ms/step - accuracy: 0.2342 - auc_6: 0.6327 - loss: 2.3117 - val_accuracy: 0.2591 - val_auc_6: 0.6720 - val_loss: 1.7884
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 86s 378ms/step - accuracy: 0.2547 - auc_6: 0.6704 - loss: 1.7859 - val_accuracy: 0.2614 - val_auc_6: 0.6558 - val_loss: 1.9045
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 78s 346ms/step - accuracy: 0.2791 - auc_6: 0.6888 - loss: 1.7499 - val_accuracy: 0.3024 - val_auc_6: 0.7087 - val_loss: 1.7206
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 76s 333ms/step - accuracy: 0.3023 - auc_6: 0.7052 - loss: 1.7222 - val_accuracy: 0.3149 - val_auc_6: 0.7177 - val_loss: 1.6901
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 76s 335ms/step - accuracy: 0.3116 - auc_6: 0.7151 - loss: 1.7018 - val_accuracy: 0.3198 - val_auc_6: 0.7248 - val_loss: 1.6861
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 74s 326ms/step - accuracy: 0.3259 - auc_6: 0.7261 - loss: 1.6762 - val_accuracy: 0.2201 - val_auc_6: 0.6310 - val_loss:

In [38]:
new_model = tf.keras.models.load_model('alexnet.keras')

TypeError: <class 'keras.src.models.sequential.Sequential'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential_5', 'trainable': True, 'dtype': 'float32', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 48, 48, 1], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_5'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d_5', 'trainable': True, 'dtype': 'float32', 'filters': 96, 'kernel_size': [11, 11], 'strides': [4, 4], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform',